In [1]:
import os
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from dotenv import load_dotenv

load_dotenv()

d:\code\multi-modal-rag-llmops\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
embedding = OpenAIEmbeddings(
    model="text-embedding-3-large",
    openai_api_key=os.getenv("MY_OPENAI_API_KEY")
)


In [3]:
def load_faiss_index(index_path="faiss_index"):
    vectorstore = FAISS.load_local(
        index_path,
        embeddings=embedding,
        allow_dangerous_deserialization=True
    )
    print("FAISS index loaded.")
    return vectorstore

In [5]:
vectorstore = load_faiss_index("../faiss_index")

FAISS index loaded.
